# Convert a series of calexp into fits files from sources Light Curve

- author Sylvie Dagoret-Campagne
- creation date 2024-05-29
- last update 2024-05-31
- affiliation : IJCLab
- Kernel **w_2024_16**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
band = 'g'
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def savecalexp(visitId,path):
    """
    inputs
      - visitid
      - path
      - collection
    """

    filename_out = f"exposure_{visitId}_calexp.fits"
    fullfilename_out=os.path.join(path,filename_out)

    datasetType = 'calexp'
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collection)

    for i, ref in enumerate(datasetRefs):
        print(ref.dataId)
        print("band:", ref.dataId['band'])
        band = ref.dataId['band']
    
    calexp = butler.get(datasetType, **dataId,collections=collections)
    calexp_info = calexp.getInfo()
    photocalib = calexp_info.getPhotoCalib()
    expo_photocalibconstant_mean = photocalib.getCalibrationMean()
    expo_photocalibconstant_error = photocalib.getCalibrationErr()
    calexp_md = calexp_info.getMetadata()
    magzero,magzero_rms,magzero_nobj = calexp_md["MAGZERO"],calexp_md["MAGZERO_RMS"],calexp_md["MAGZERO_NOBJ"] 
    bgmean,bgvar = calexp_md["BGMEAN"],calexp_md["BGVAR"]

    mask = calexp.mask.array
    the_mask = np.where(mask==0,1,0)

    img = calexp.image.array 
    wcs = calexp.getWcs()
    # Convert WCS-DM to astropy WCS
    the_fits_WCS = WCS(wcs.getFitsMetadata())

    # create the header
    header = the_fits_WCS.to_header()

    md = calexp_md.toDict() 

    for key,value in md.items():
    # DS9 cannot read long cards    
        if len(key)<8:
            header[str(key)] = value
 
    primary_hdu = fits.PrimaryHDU(header=header)
    image_hdu = fits.ImageHDU(img)
    hdu_list = fits.HDUList([primary_hdu, image_hdu])
    hdu_list.writeto(fullfilename_out,overwrite=True)    

    return header


## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
#inputfilename = "sources_objectTable-t3864-bg-o547-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
inputfilename = "sources_objectTable-t3864-bg-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
df_myselectedvisits = pd.read_csv(inputfilename ,index_col=0)
tract = 3864
band = "g"
objectname = 912
path = f"calexp_t{tract}_b{band}_o{objectname}"
title = f"Auxtel Light Curves : tract = {tract}, band = {band}, object = {objectname} "
suptitle = inputfilename 

In [ ]:
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
df_myselectedvisits.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="psfMag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
print(path)

In [ ]:
listOfVisitId = df_myselectedvisits["visit"].values
listOfVisitId

In [ ]:
all_headers = []
for visit in listOfVisitId:
    h = savecalexp(visit,path)
    all_headers.append(h) 
    